<center><h1>Tabular Playground - October 2021</h1><center>
    <center><h2>EDA, Optuna/XGB Implementation</h2><center>
        <center><h3>By Tariq Hussain</center></h3>

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
def reduce_mem_usage(df):
    #iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    #create a dataframe and optimize its memory usage
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:
train_fp = '../input/tabular-playground-series-oct-2021/train.csv'
train_df = import_data(train_fp)

test_fp = '../input/tabular-playground-series-oct-2021/test.csv'
test_df = import_data(test_fp)


In [ ]:
train_df

In [ ]:
train_df.info()

In [ ]:
train_df.dtypes

In [ ]:
print(train_df.isnull().values.any())
print(' ')
print(train_df.isnull().sum())

In [ ]:
train = train_df.drop(['id'], axis=1)

In [ ]:
test_df

In [ ]:
print(test_df.isnull().values.any())
print(' ')
print(test_df.isnull().sum())

In [ ]:
test_df = test_df.drop(['id'], axis=1)

In [ ]:
train_corr = train_df.corr()
mask = np.triu(np.ones_like(train_corr, dtype=np.bool))

fig = plt.figure(figsize=(16,13))
sns.heatmap(train_corr, mask=mask)
plt.title('Correlation between features')

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

train_copy = train.copy()

X = train_copy.drop(['target'], axis=1)
y = train_copy.target


In [ ]:
X

In [ ]:
#scaler = MinMaxScaler()

test = test_df.copy()

#X_scaled = scaler.fit_transform(X)

In [ ]:
#X_scaled = pd.DataFrame(X_scaled)

In [ ]:
#X_scaled

In [ ]:
X['std'] = X.std(axis=1)
X['min'] = X.min(axis=1)
X['max'] = X.max(axis=1)

test['std'] = test.std(axis=1)
test['min'] = test.min(axis=1)
test['max'] = test.max(axis=1)

In [ ]:
display(X)

In [ ]:
display(test)

In [ ]:
'''
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import log_loss, accuracy_score, mean_absolute_error, r2_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score

def objective(trial):
    
    xtrain,xvalid,ytrain,yvalid = train_test_split(X,y,test_size=0.2,random_state=0)
    
    learning_rate = trial.suggest_loguniform('learning_rate', 0.00001, 10.0)
    max_depth = trial.suggest_int('max_depth', 4, 10)
    l1_reg = trial.suggest_loguniform('l1_reg', 0.00001, 10.0)
    l2_reg = trial.suggest_loguniform('l2_reg', 0.00001, 10.0)
    
    params = {    
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'l1_reg': l1_reg,
        'l2_reg': l2_reg
    }
    
    model = XGBRegressor(**params,
                        tree_method='gpu_hist',
                        gpu_id=0,
                        predictor='gpu_predictor'
                        )

    model.fit(
        xtrain, 
        ytrain,
        early_stopping_rounds=10,
        eval_set=[(xvalid, yvalid)], 
        verbose=False
    )

    yhat = model.predict(xvalid)
    return roc_auc_score(yvalid, yhat)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)
print(study.best_params)
'''

In [ ]:
import optuna
from optuna.samplers import TPESampler
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from xgboost import XGBClassifier
from sklearn.metrics import log_loss, accuracy_score, mean_absolute_error, r2_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from optuna.integration import XGBoostPruningCallback

def objective(trial):
    xtrain,xvalid,ytrain,yvalid = train_test_split(X,y,test_size=0.2,random_state=0)
    
    params = {
        'max_depth': trial.suggest_int('max_depth', 6, 10),
        #'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 400, 4000, 400), 
        'eta': trial.suggest_float('eta', 0.007, 0.013), 
        'subsample': trial.suggest_discrete_uniform('subsample', 0.2, 0.9, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.2, 0.9, 0.1),
        'colsample_bylevel': trial.suggest_discrete_uniform('colsample_bylevel', 0.2, 0.9, 0.1),
        'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-4, 1e4), 
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-4, 1e4),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 1e4), 
        'gamma': trial.suggest_loguniform('gamma', 1e-4, 1e4),
        'eval_metric' : 'auc',
        'objective' : 'binary:logistic',
        'gpu_id': 0,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor'
            }
    
    model = XGBClassifier(**params,
                         use_label_encoder=False)

    model.fit(
        xtrain, 
        ytrain,
        early_stopping_rounds=100,
        eval_set=[(xvalid, yvalid)], 
        verbose=False
    )

    yhat = model.predict(xvalid)
    return roc_auc_score(yvalid, yhat)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)
print(study.best_params)

In [ ]:
print("Best value (RMSE): {:.5f}".format(study.best_value))
print("Best params:")

for key, value in study.best_params.items():
    print("{}: {}".format(key, value))

In [ ]:
xgb_params = study.best_params
xgb_params

In [ ]:
xtrain,xvalid,ytrain,yvalid = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
xgb = XGBClassifier(**xgb_params,
                    learning_rate=0.02,
          tree_method='gpu_hist', 
          predictor='gpu_predictor', 
          eval_metric='auc',
          gpu_id=0,
         random_state=0,
                   use_label_encoder=False)

xgb.fit(
    xtrain, 
    ytrain,
    early_stopping_rounds=100,
    eval_set=[(xvalid, yvalid)], 
    verbose=False
    )

In [ ]:
predictions = xgb.predict(test)

In [ ]:
final_output = np.mean(np.column_stack(predictions), axis=0)

In [ ]:
sample_fp = '../input/tabular-playground-series-oct-2021/sample_submission.csv'

sample = pd.read_csv(sample_fp)

In [ ]:
submission = sample.copy()

submission['target'] = final_output

submission


In [ ]:
submission.to_csv('submission.csv', index=False)